## 1. Data import

Import the simulated data and convert in into 3x3x2 ndarray. 3x3 is the tic-tac-toe grid dimensions. 2 are the values of X and O. 1-0 represents X, 0-1 represents O, 0-0 represents empty cell.

Input data representation for the model is:
* Input: current state of 3x3x2 (board)
* Output(Int 1-9) : Position of next move to make

In [331]:
import numpy as np

In [332]:
input_file = 'good_games_data.csv'
raw_data = np.genfromtxt(input_file, delimiter=',')
print("Length of raw data:", len(raw_data))
print("Sample row:", raw_data[0])

Length of raw data: 15119
Sample row: [1. 2. 3. 4. 5. 6. 7. 0. 0. 1.]


In [333]:
def playNextMove(board, move, player):
    playerIndex = player - 1
    moveIndex = move - 1
    
    r, c = indexToRowCol(moveIndex)
    board[r, c, playerIndex] = 1

def indexToRowCol(index):
    r = index // 3
    c = index % 3
    return r, c

In [334]:
def convertRawToModelInput(raw_data):
    X = []
    Y = []
    for raw in raw_data:
        board = np.zeros((3,3,2))
        player = 0
        for i in range(9):
            if raw[i] == 0:
                break #game already over

            if raw[-1] == 1:
                if i%2 == 0:
                    player = 1
                    X.append(board.copy())
                    Y.append(raw[i])
                else:
                    player = 2

            else:           
                if i%2 == 1:
                    player = 2
                    X.append(board.copy())
                    Y.append(raw[i])
                else:
                    player = 1

            playNextMove(board, int(raw[i]), player)

    return np.array(X), np.array(Y)


In [335]:
def shuffle_in_unison(a, b):
     n_elem = a.shape[0]
     indeces = np.random.choice(n_elem, size=n_elem, replace=False)
     return a[indeces], b[indeces]

In [336]:
def convertToOneHot(Y, n_y):
    Y = Y.astype(int)
    Y_index = Y - 1  # convert position to index
    m = len(Y)
    Y_oh = np.zeros((m, n_y))
    Y_oh[np.arange(m), Y_index] = 1
    return Y_oh

In [337]:
X, Y = convertRawToModelInput(raw_data)
X,Y = shuffle_in_unison(X, Y)
Y = convertToOneHot(Y, 9)
print("Size of input data:", len(Y))

Size of input data: 53792


In [338]:
m = 50000
X_train = X[:m]
Y_train = Y[:m]
X_test = X[m:]
Y_test = Y[m:]

print("Shape of X_train", X_train.shape)
print("Shape of Y_train", Y_train.shape)
print("Shape of X_test", X_test.shape)
print("Shape of Y_test", Y_test.shape)

Shape of X_train (50000, 3, 3, 2)
Shape of Y_train (50000, 9)
Shape of X_test (3792, 3, 3, 2)
Shape of Y_test (3792, 9)


## 2. Model

Return instance of the model

In [339]:
from keras.models import Model
from keras.layers import Input, Dense, Conv2D, Flatten
from keras.optimizers import Adam

In [340]:
def model():
    
    X_train = Input(shape=(3,3,2,))
    X = Conv2D(128, (3,3), input_shape=(3, 3, 2), activation="relu")(X_train)
    X = Flatten()(X)
    X = Dense(128, activation="relu")(X)
    X = Dense(64, activation="relu")(X)
    X = Dense(32, activation="relu")(X)
    Y = Dense(9, activation="softmax")(X)
    
    model = Model(inputs=X_train, outputs= Y)
    return model

In [341]:
m = model()
m.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_19 (InputLayer)        (None, 3, 3, 2)           0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 1, 1, 128)         2432      
_________________________________________________________________
flatten_7 (Flatten)          (None, 128)               0         
_________________________________________________________________
dense_45 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_46 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_47 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_48 (Dense)             (None, 9)                 297       
Total para

In [342]:
opt = Adam()
m.compile(opt, loss="categorical_crossentropy", metrics=['accuracy'])

In [344]:
m.fit(X_train, Y_train, epochs=10, batch_size=32)

Epoch 1/10
50000/50000 [==============================] - 5s 99us/step - loss: 1.3363 - acc: 0.4309
Epoch 2/10
50000/50000 [==============================] - 5s 97us/step - loss: 1.3340 - acc: 0.4301
Epoch 3/10
50000/50000 [==============================] - 5s 97us/step - loss: 1.3295 - acc: 0.4328
Epoch 4/10
50000/50000 [==============================] - 5s 97us/step - loss: 1.3271 - acc: 0.4330
Epoch 5/10
50000/50000 [==============================] - 5s 98us/step - loss: 1.3253 - acc: 0.4325
Epoch 6/10
50000/50000 [==============================] - 6s 110us/step - loss: 1.3247 - acc: 0.4308
Epoch 7/10
50000/50000 [==============================] - 5s 108us/step - loss: 1.3222 - acc: 0.4320
Epoch 8/10
50000/50000 [==============================] - 5s 102us/step - loss: 1.3196 - acc: 0.4337
Epoch 9/10
50000/50000 [==============================] - 5s 101us/step - loss: 1.3205 - acc: 0.4299
Epoch 10/10
50000/50000 [==============================] - 5s 100us/step - loss: 1.3181 - acc: 0

In [345]:
m.evaluate(X_test, Y_test, batch_size=32)

3792/3792 [==============================] - 0s 90us/step


[1.3253529594920355, 0.42273206751054854]

## 3. Test

In [346]:
def printBoard(board):
    def getChar(a):
        if np.array_equal(np.array([1,0]), a):
            return 'X'
        elif np.array_equal(np.array([0,1]), a):
            return 'O'
        return '-'

    for i in range(3):
        s = []
        for j in range(3):
            s.append(getChar(board[i][j]))
        print(" ".join(s))

In [375]:
board = np.zeros((3,3,2))

In [380]:
print("Input board:")
printBoard(board)

# model predict
input = np.empty((1,3,3,2))
input[0] = board
output = m.predict(input)
moveIndex = np.argmax(output)

# update board
playerIndex = int(board.sum() % 2)
r, c = indexToRowCol(moveIndex)
board[r, c, playerIndex] = 1
print("Output board:")
printBoard(board)

Input board:
- O X
- X -
O - -
Output board:
- O X
X X -
O - -


In [379]:
player = 1   # 0: X, 1: O
my_move = 6
r, c = indexToRowCol(my_move)
board[r,c,player] = 1
